# Email Mindmap Demo (Week 5 Community Contribution)

Welcome to the **Email Mindmap Demo** notebook! This demo walks you through a workflow for exploring and visualizing email relationships using embeddings and mindmaps.

---

## 📋 Workflow Overview

1. **Load/Create Synthetic Email Data**  
   Generate or load varied types of emails: work, personal, family, subscriptions, etc.

2. **Generate Embeddings**  
   Use an open-source model to create vector embeddings for email content.

3. **Build & Visualize a Mindmap**  
   Construct a mindmap of email relationships and visualize it interactively using `networkx` and `matplotlib`.

4. **Question-Answering Interface**  
   Query the email content and the mindmap using a simple Q&A interface powered by Gradio.

---

## ⚙️ Requirements

> **Tip:**  
> I'm including an example of the synthetic emails in case you don't want to run that part.
> Might need to install other libraries like pyvis, nbformat and faiss-cpu


## ✨ Features

- Synthetic generation of varied emails (work, personal, family, subscriptions)
- Embedding generation with open-source models (hugging face sentence-transformer)
- Interactive mindmap visualization (`networkx`, `pyvis`)
- Simple chatbot interface (Gradio) and visualization of mindmap created

---



In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if ollama_api_key:
    print(f"OLLAMA API Key exists and begins {ollama_api_key[:2]}")
else:
    print("OLLAMA API Key not set (and this is optional)")

# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key=ollama_api_key, base_url=ollama_url)



## Preparation of synthetic data (could have been week2 work)

In [ ]:
#using ollama gpt oss 120b cloud i'm going to create synthetic emails using a persona.
#they are going to be saved in a json file with different keys
from pydantic import BaseModel, Field
from typing import List, Optional


class Email(BaseModel):
    sender: str = Field(description="Email address of the sender")
    subject: str = Field(description="Email subject line")
    body: str = Field(description="Email body content")
    timestamp: str = Field(description="ISO 8601 timestamp when email was received")
    category: str = Field(description="Category of the email")

class EmailBatch(BaseModel):
    emails: List[Email] = Field(description="List of generated emails")


In [ ]:
def create_persona(name: str, age: int, occupation: str, 
                   interests: List[str], family_status: str) -> str:
    persona = f"""
    You are generating synthetic emails for a realistic inbox simulation.

    **Person Profile:**
    - Name: {name}
    - Age: {age}
    - Occupation: {occupation}
    - Interests: {', '.join(interests)}
    - Family Status: {family_status}

    **Email Categories to Include:**
    1. **Work Emails**: Project updates, meeting invitations, colleague communications, 
    performance reviews, company announcements
    2. **Purchases**: Order confirmations, shipping notifications, delivery updates, 
    receipts from various retailers (Amazon, local shops, etc.)
    3. **Subscriptions**: Newsletter updates, streaming services (Netflix, Spotify), 
    software subscriptions (Adobe, Microsoft 365), magazine subscriptions
    4. **Family**: Communications with parents, siblings, children, extended family members,
    family event planning, photo sharing
    5. **Friends**: Social plans, birthday wishes, casual conversations, group hangouts,
    catching up messages
    6. **Finance**: Bank statements, credit card bills, investment updates, tax documents,
    payment reminders
    7. **Social Media**: Facebook notifications, LinkedIn updates, Instagram activity,
    Twitter mentions
    8. **Personal**: Doctor appointments, gym memberships, utility bills, insurance updates

    **Instructions:**
    - Generate realistic email content that reflects the person's life over time
    - Include temporal patterns (more work emails on weekdays, more personal on weekends)
    - Create realistic sender names and email addresses
    - Vary email length and formality based on context
    - Include realistic subject lines
    - Make emails interconnected when appropriate (e.g., follow-up emails, conversation threads)
    - Include seasonal events (holidays, birthdays, annual renewals)
    """
    return persona

persona_description = create_persona(
    name="John Doe",
    age=30,
    occupation="Software Engineer",
    interests=["technology", "reading", "traveling"],
    family_status="single"
)


In [ ]:
from openai import OpenAI
from datetime import datetime, timedelta
import random
from typing import List

def generate_synthetic_emails(
    persona_description: str,
    num_emails: int,
    start_date: str,
    end_date: str,
    model: str = "gpt-4o-2024-08-06"
) -> List[Email]:
    """
    NEEDS TO WORK WITH OPENAI MODELS BECAUSE OF PARSED (STRUC OUTPUT) MODELS
    Generates synthetic emails using OpenAI's structured output feature.
    
    Args:
        persona_description: Detailed persona description
        num_emails: Number of emails to generate per batch
        start_date: Start date for email timestamps
        end_date: End date for email timestamps
        model: OpenAI model to use (must support structured outputs)
    
    Returns:
        List of Email objects
    """
    
    # Calculate date range for context
    date_range_context = f"""
        Generate emails with timestamps between {start_date} and {end_date}.
        Distribute emails naturally across this time period, with realistic patterns:
        - More emails during business hours on weekdays
        - Fewer emails late at night
        - Occasional weekend emails
        - Bursts of activity around events or busy periods
        """
            
            # System message combining persona and structure instructions
    system_message = f"""
        {persona_description}

        {date_range_context}

        Generate {num_emails} realistic emails that fit this person's life. 
        Ensure variety in categories, senders, and content while maintaining realism.
        """
    
    try:
        client = OpenAI()

        response = client.chat.completions.parse(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": system_message
                },
                {
                    "role": "user",
                    "content": f"Generate {num_emails} diverse, realistic emails for this person's inbox."
                }
            ],
            response_format=EmailBatch,
        )
        return response.choices[0].message.parsed.emails
    
    except Exception as e:
        print(f"Error generating emails: {e}")
        return []


def save_emails_to_json(emails: List[Email], filename: str):
    """
    Saves emails to a JSON file.
    """
    import json
    
    emails_dict = [email.model_dump() for email in emails]
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(emails_dict, f, indent=2, ensure_ascii=False)
    
    print(f"Saved {len(emails)} emails to {filename}")


In [ ]:
mails_2 = generate_synthetic_emails(
    persona_description = persona_description,
    num_emails = 100,
    start_date = '2024-06-01',
    end_date = '2025-01-01',
    model = "gpt-4o"
    )

In [ ]:
save_emails_to_json(mails_2, 'emails2.json')

## Create embeddings for the mails



In [ ]:
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
import json
from langchain.vectorstores import FAISS

#MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [ ]:
# Read in emails from the emails.json file and construct LangChain documents


with open("emails.json", "r", encoding="utf-8") as f:
    emails = json.load(f)

documents = []
for email in emails:
    # Extract metadata (all fields except 'content')
    metadata = {k: v for k, v in email.items() if k in ['sender','category','timestamp']}
    body = email.get("body", "")
    documents.append(Document(page_content=body, metadata=metadata))

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Sample metadata fields: {list(documents[0].metadata.keys()) if documents else []}")

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings_model).delete_collection()

vectorstore = FAISS.from_documents(chunks, embedding=embeddings_model)

all_embeddings = [vectorstore.index.reconstruct(i) for i in range(vectorstore.index.ntotal)]

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d


## Visualizing mindmap

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE  # Or use UMAP
from pyvis.network import Network

# Here, emails is your list of email objects, with .subject or .body

# Build similarity graph
def build_mindmap_html(emails, all_embeddings, threshold=0.6):
    similarity = cosine_similarity(all_embeddings)

    G = nx.Graph()
    for i, email in enumerate(emails):
        G.add_node(i, label=email['subject'][:80], title=email['body'][:50])  # Custom hover text

    for i in range(len(emails)):
        for j in range(i+1, len(emails)):
            if similarity[i][j] > threshold:
                G.add_edge(i, j, weight=float(similarity[i][j]))

    # Convert to pyvis network
    nt = Network(notebook=True, height='700px', width='100%', bgcolor='#222222', font_color='white')
    nt.from_nx(G)
    html = nt.generate_html().replace("'", "\"")
    return html


## Putting it all together in a gradio.
It needs to have an interface to make questions, and the visual to see the mindmap.


In [ ]:
# create a new Chat with OpenAI
MODEL="gpt-4o-mini"
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()
from langchain_core.callbacks import StdOutCallbackHandler

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain_debug = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:

import gradio as gr

def show_mindmap():
    # Call build_mindmap_html to generate the HTML
    html = build_mindmap_html(emails, all_embeddings)
    return f"""<iframe style="width: 100%; height: 600px;margin:0 auto" name="result" allow="midi; geolocation; microphone; camera; 
    display-capture; encrypted-media;" sandbox="allow-modals allow-forms 
    allow-scripts allow-same-origin allow-popups 
    allow-top-navigation-by-user-activation allow-downloads" allowfullscreen="" 
    allowpaymentrequest="" frameborder="0" srcdoc='{html}'></iframe>"""


with gr.Blocks(title="Mindmap & Email Chatbot") as demo:
    gr.Markdown("# 📧 Mindmap Visualization & Email QA Chatbot")
    with gr.Row():
        chatbot = gr.ChatInterface(fn=chat, title="Ask about your emails",
            examples=[
                    "What is my most important message?",
                    "Who have I been communicating with?",
                    "Summarize recent emails"
                ],
)
        mindmap_html = gr.HTML(
            show_mindmap,
            label="🧠 Mindmap of Your Emails",
        )
        # Reduce height: update show_mindmap (elsewhere) to ~400px, or do inline replace for the demo here:
        # mindmap_html = gr.HTML(lambda: show_mindmap().replace("height: 600px", "height: 400px"))
            
demo.launch(inbrowser=True)

